## Streamlit

In [ ]:
!pip install streamlit
!pip install haversine
!pip install airportsdata
!pip install mealpy

In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.771s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile my_app.py

import streamlit as st
from PIL import Image
from datetime import datetime
import time
import random

import pandas as pd
import numpy as np
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from numpy import array


from mealpy import PSO, ACOR, GA
import airportsdata
from mealpy import PermutationVar, Problem
from haversine import haversine, Unit

from MetaHeuristics import Meta
from initial import air_predictor
s

## initialize variables
LSTM_model = load_model("/content/drive/Shareddrives/Flight Prediction/Miscellaneous/LSTM_model_final.h5")
df = pd.read_csv('/content/drive/Shareddrives/Flight Prediction/Miscellaneous/flights.csv')


future = pd.read_csv('/content/drive/Shareddrives/Flight Prediction/Miscellaneous/TEST.csv')
future_data = future[['FlightDate', 'Month', 'Day', 'DayOfWeek']]
future = future[['FlightDate', 'Month', 'Day', 'DayOfWeek']]

label_encoder = LabelEncoder()

for column in future.columns:
    if future[column].dtype == 'object':
        future[column] = label_encoder.fit_transform(future[column])

scaler = StandardScaler()
scaler.fit(future)
X_scaled = scaler.transform(future)

X_samples = [X_scaled[i:i + 3] for i in range(len(X_scaled) - 3 + 1)]

X = np.array(X_samples)
X = X.reshape(-1, 3, 4)

y_pred = LSTM_model.predict(X)
y_pred = np.round(y_pred, 2)

predict_travels = y_pred

future_df = pd.DataFrame(y_pred.flatten(), columns=['Predicted_Delay'])
future_pred = pd.concat([future_data, future_df], axis=1)
future_pred.dropna(inplace=True)
future_pred['FlightDate'] = pd.to_datetime(future_pred['FlightDate'])
prediction = future_pred[['FlightDate', 'Predicted_Delay']]


date_range = pd.date_range(start='2021-08-01', end='2024-12-29')
date_range = date_range.strftime('%Y-%m-%d')


def get_coordinates(geo_loc):
    airports = airportsdata.load('IATA')

    if geo_loc not in airports:
        return 0, 0
    return airports[geo_loc]['lat'], airports[geo_loc]['lon']

## metaheu
meta_model = GA.BaseGA
params = {
          'model': meta_model,
          'epoch': 100,
          'pop_size': 20
          }


# Streamlit-------------------

st.title('airlines_in_maNILa')

# Opening the image
image = Image.open('/content/drive/Shareddrives/Flight Prediction/Miscellaneous/cat-flight.png')
st.image(image, width=400)

st.header('Please enter your flight information:')

start_date = datetime(2021, 1, 1)
end_date = datetime(2024, 12, 29)

run_air = air_predictor(df, [date_range, predict_travels])
waypoint = run_air.separate_strings_with_dash()

flight_date = str(st.date_input('Flight Date:', min_value=start_date, max_value=end_date, format='YYYY-MM-DD'))

origin = st.selectbox('Origin Airport Code:', waypoint)
destination = st.selectbox('Destination Airport Code:', waypoint)

def is_delay(get_df, flight_date):
    if get_df[get_df['FlightDate'] == flight_date]['Predicted_Delay'].values >= 15:
        return True
    else:
        return False


if st.button('Predict Chosen Flight Details for Possible Delay'):
    st.subheader("Hep Hep! We, airlines in maNILa, are currently working on it...")
    get_delay = is_delay(prediction, flight_date)
    run_air.choose_delay(flight_date)
    start = run_air.choose_orig(origin)
    end = run_air.choose_dest(destination)

    start_date, end_date = run_air.date_parse()

    coor = [get_coordinates(x) for x in waypoint]
    maps = dict(zip(waypoint, coor))

    router = Meta(maps, start, end, params)

    # Add a placeholder for progress
    progress_placeholder = st.empty()
    bar = progress_placeholder.progress(0)

    # Perform computation
    route, dist, orig_dist = router.run_meta()

    # Update progress bar
    for i in range(100):
        bar.progress(i + 1)
        time.sleep(0.1)

    opt_route = ' -> '.join(route)
    orig_route = f'{route[0]} -> {route[len(route)-1]}'

    dt = datetime.now()
    no_delay = prediction.query(f'Predicted_Delay < 15.0 and FlightDate >= "{start_date}" and FlightDate <= "{end_date}"')


    if get_delay ==  True:
        # Update placeholders with results
        st.write('Hi! Unfortunately, your flight on', flight_date, 'is predicted to have a delay however,')
        st.write('Here are the alternative routes for your chosen destination:', opt_route)
        st.write('Above are the routes to experience possible little to no delays.')
        st.write('Computed distance:', dist, 'km')
        st.write('Other dates with little to no delay')

        st.dataframe(prediction['FlightDate'].dt.strftime('%Y-%m-%d'))

        progress_placeholder.empty()  # Clear the progress bar

        st.subheader("...Success! Here are your recommendations (❁´▽`❁)")
    else:
        # Update placeholders with results
        st.write('Hi! fortunately, your flight on', flight_date, 'is predicted to have no delays, Congratulations!')
        st.write('You can fly driectly from', start, 'to', end, 'with the possibility of experiencing little to no delays.')
        st.write('Computed distance:', orig_dist)

        progress_placeholder.empty()  # Clear the progress bar

        st.subheader("...Success! Here are your recommendations (❁´▽`❁)")

st.subheader("Kindly support our web-app application by using our recommendation system often! (It's reliable, source: kutob at panalangin)")


Overwriting my_app.py


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.73.244.8


In [ ]:
!streamlit run my_app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.146s
your url is: https://yellow-otters-walk.loca.lt


<br><br><br>